In [ ]:
# using GuSTO
include("../src/GuSTO.jl")

In [ ]:
robot = PandaBot()
env = Factory()

push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.25,0.30), Vec3f0(.3,0.5,0.01)))
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.25,0.62), Vec3f0(.3,0.5,0.01)))

push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,-0.26,0.30), Vec3f0(.3,0.01,0.33)))
push!(env.obstacle_set, HyperRectangle(Vec3f0(0.45,0.25,0.30), Vec3f0(.3,0.01,0.33)))

th_init = [0;  -0.6; 0.; 1.20*pi; 0.25; 1.10*pi; 0.25*pi+0.001]
th_goal = [0;  +0.1; 0.; 1.20*pi; 0.25; 1.42*pi; 0.25*pi-0.001]

In [ ]:
pan = Panda()

state = RigidBodyDynamics.MechanismState(pan.mechanism)
world_frame = RigidBodyDynamics.root_frame(pan.mechanism)

set_configuration!(state,th_goal)

EE_id = 7 + 2
EE_link = RigidBodyDynamics.bodies(pan.mechanism)[EE_id]
EE_link_frame = RigidBodyDynamics.default_frame(EE_link)
EE_link_point = RigidBodyDynamics.Point3D(EE_link_frame, 0.0,0.0,0.)
p_EE = RigidBodyDynamics.Spatial.transform(state,EE_link_point,world_frame)
p_EE.v

world_frame = RigidBodyDynamics.root_frame(pan.mechanism)
EE_path = RigidBodyDynamics.path(pan.mechanism, root_body(pan.mechanism), EE_link)
J_pEE_joint = point_jacobian(state,EE_path, RigidBodyDynamics.Spatial.transform(state,EE_link_point,world_frame))
J_pEE_joint.J[:,1:robot.num_joints];

In [ ]:
model = PandaKin()

x_init, x_goal = zeros(model.x_dim), zeros(model.x_dim)
for i in 1:robot.num_joints
    x_init[2*i-1:2*i] = [cos(th_init[i]); sin(th_init[i])]
    x_goal[2*i-1:2*i] = [cos(th_goal[i]); sin(th_goal[i])]
end

r_EE_inWorldFrame = get_EE_position(x_goal,robot) + [0.01;-0.02;0.02]
model.p_EE_goal = r_EE_inWorldFrame

N = 81
tf_guess = 100.

In [ ]:
PD = ProblemDefinition(robot, model, env, x_init, x_goal);
TOP = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOS_SCP = TrajectoryOptimizationSolution(TOP)

solve_SCP!(TOS_SCP, TOP, solve_gusto_jump!, init_traj_so1, "Ipopt", print_level=0)

In [ ]:
get_joint_position(x_init,robot,7)

In [ ]:
@show TOS_SCP.SCPS.converged
@show TOS_SCP.SCPS.successful
@show TOS_SCP.SCPS.iterations
@show TOS_SCP.SCPS.total_time
@show TOS_SCP.SCPS.accept_solution
@show TOS_SCP.SCPS.prob_status
@show TOS_SCP.SCPS.convergence_measure
@show TOS_SCP.SCPS.param.alg.ω_vec
@show TOS_SCP.SCPS.param.alg.Δ_vec
@show TOS_SCP.SCPS.J_true[end];

In [ ]:
r_EE_f = get_EE_position(TOS_SCP.traj.X[:,end],robot)
println("Distance between r_EE_goal and solution is ", norm(r_EE_f-r_EE_inWorldFrame))
abs.(r_EE_f-r_EE_inWorldFrame)
[r_EE_f r_EE_inWorldFrame]

In [ ]:
joint_traj = zeros(robot.num_joints,N)
for k in 1:N
    joint_traj[:,k] = get_configuration(TOS_SCP.traj.X[:,k],model)
end
verify_joint_limits(TOS_SCP.SCPS.traj, TOS_SCP.SCPS.SCPP)

In [ ]:
using MeshCat
using MeshCatMechanisms

using RigidBodySim
using RigidBodyDynamics

vis = Visualizer()

pd = robot.pan
mvis = MechanismVisualizer(
    pd.mechanism,
    URDFVisuals(PandaRobot.urdfpath(), package_path=[dirname(dirname(PandaRobot.urdfpath()))]),
    vis);
# 
vis[:obstacles]
for (idx,obs) in enumerate(env.obstacle_set)
    setobject!(vis[:obstacles][Symbol(string("obs",idx))], 
        Object(obs,MeshBasicMaterial(color=RGBA(0.,0.,0.,0.5))))
end

EE_box = HyperRectangle(Vec3f0(model.p_EE_goal.-model.p_EE_goal_delta_error), Vec3f0(2*model.p_EE_goal_delta_error*ones(3)))
setobject!(vis[:obstacles][Symbol(string("obs",length(env.obstacle_set)+1))], 
        Object(EE_box,MeshBasicMaterial(color=RGBA(1.,1.,1.,0.5))))

q = Vector{Array{Float64,1}}(0)
for k in 1:N
    push!(q, joint_traj[:,k]) 
end

setanimation!(mvis, 1:length(q), q)

plot_in_cell = true
plot_in_cell ? IJuliaCell(vis) : open(vis)
#sleep(1)